In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!unzip "/content/drive/MyDrive/Portugues/modelo_fakenews_final.zip" -d "/content/modelo_fakenews_final"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

modelo_dir = "/content/modelo_fakenews_final/modelo_fakenews_final/checkpoint-7450"

# Carrega o tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelo_dir, model_type="bert")

# Carrega o modelo treinado
model = AutoModelForSequenceClassification.from_pretrained(modelo_dir)

# Coloca em modo de avaliação
model.eval()


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


modelo_dir = "/content/modelo_fakenews_final/modelo_fakenews_final/checkpoint-7450"
csv_path = "/content/drive/MyDrive/Portugues/test_dataset.csv"

# Define o dispositivo de execução GPU (cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carrega o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained(modelo_dir, model_type="bert")
model = AutoModelForSequenceClassification.from_pretrained(modelo_dir).to(device)
model.eval()  #modo de avaliação

# Lê o dataset de teste
df_test = pd.read_csv(csv_path, sep=";", encoding="utf-8")

# F faz previsões em lotes p evitar sobrecarga de memória
def predict_in_batches(texts, batch_size=200):
    preds = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        # Desativa o cálculo  gradiente e gera previsões
        with torch.no_grad():
            outputs = model(**inputs)
            batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            preds.extend(batch_preds)

    return preds


print("Gerando predições")
preds = predict_in_batches(df_test["texto"].tolist(), batch_size=200)

# Calcula métricas de desempenho
y_true = df_test["label"].values
print("\nAcurácia:", round(accuracy_score(y_true, preds), 4))
print("\nRelatório de Classificação:")
print(classification_report(y_true, preds, target_names=["FAKE", "REAL"]))

# Identifica e mostra as amostras que o modelo errou
df_test["predito"] = preds
erros = df_test[df_test["predito"] != df_test["label"]]
print(f"\nTotal de erros: {len(erros)} de {len(df_test)} amostras")

# Cria uma matriz de confusão
cm = confusion_matrix(y_true, preds)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Falso Positivo", "Verdadeiro Positivo"],
            yticklabels=["Verdadeiro Negativo", "Falso Negativo"])
plt.title("Matriz de Confusão - BERTimbau")
plt.xlabel("Previsão do Modelo")
plt.ylabel("Valor Real")
plt.show()

# Salva as predições completas
output_path = "/content/drive/MyDrive/Portugues/resultados_bert.csv"
df_test.to_csv(output_path, sep=";", index=False, encoding="utf-8-sig")
print(f"Arquivo de predições salvo em: {output_path}")
